# Libraries

In [19]:
import requests
import pandas as pd
import json
import os

# Basic API Setup

********_NOTE_********

An API query to the Census Bureau in the search bar of your browser is one way to pull this / other data.
But it's frustrating for repeated or multi-step data pulls, e.g. getting data across several years.
For reference, the address below is an example of a browser search bar API query:

https://api.census.gov/data/2022/acs/acs1/subject?get=NAME,S0801_C01_001E&for=state:26

In [20]:
# Set working directory
os.chdir('/Users/grantschwab/Desktop/Detroit News/Data')

# Import API key from stored txt file
with open('./census_api_key.txt', 'r') as file:
    api_key = file.read().strip()

# Define the base URL for the Census API
base_url = 'https://api.census.gov/data/'

# Choose vars, loc, years

In [21]:
#Specify the variables you want to pull
variables = [
    'S0801_C01_001E',
    'S0801_C01_002E',
    'S0801_C01_009E',
    'S0801_C01_010E',
    'S0801_C01_011E',
    'S0801_C01_012E',
    'S0801_C01_013E']

# Specify the state code for Michigan (FIPS code)
state_code = '26'

# Specify the range of years
years = range(2018, 2023)

# Make list for rename cols

In [22]:
# Descriptive names for variables
variable_names = [
    'total_workers', 
    'private_car',
    'public_transit',
    'walk',
    'bike',
    'taxi_motorcycle_other',
    'work_from_home']

# API pull, DF creation

In [23]:
# Create an empty DataFrame to store the results
result_df = pd.DataFrame()

# Iterate over each year and pull data for the specified variables
for year in years:
    # Construct the API endpoint
    endpoint = f'{base_url}{year}/acs/acs1/subject?'
    
    # Specify parameters for the API request
    params = {
        'get': ','.join(variables),
        'for': f'state:{state_code}',
        'key': api_key,
    }

    # Make the API request
    response = requests.get(endpoint, params=params)

    try:
        # Try to decode the JSON response
        data = response.json()

        # Convert the data to a DataFrame
        year_df = pd.DataFrame(data[1:], columns=data[0])

        # Add a column for the year
        year_df['Year'] = year

        # Rename the columns based on the descriptive names
        year_df.rename(columns=dict(zip(variables, variable_names)), inplace=True)

        # Concatenate the results to the main DataFrame
        result_df = pd.concat([result_df, year_df], ignore_index=True)
        
    except json.decoder.JSONDecodeError:
        # Handle JSON decoding error
        print(f'Error decoding JSON for {year}: {response.text}')

Error decoding JSON for 2020: <!doctype html><html lang="en"><head><title>HTTP Status 404 ? Not Found</title><style type="text/css">body {font-family:Tahoma,Arial,sans-serif;} h1, h2, h3, b {color:white;background-color:#525D76;} h1 {font-size:22px;} h2 {font-size:16px;} h3 {font-size:14px;} p {font-size:12px;} a {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status 404 ? Not Found</h1></body></html>


In [24]:
#Preview
result_df

,total_workers,private_car,public_transit,walk,bike,taxi_motorcycle_other,work_from_home,state,Year
0,4615506,90.9,1.4,2.3,0.4,0.9,4.1,26,2018
1,4662155,90.7,1.4,2.2,0.3,0.9,4.5,26,2019
2,4526178,79.7,0.8,1.8,0.3,1.0,16.4,26,2021
3,4706068,81.9,0.8,2.0,0.4,1.2,13.7,26,2022


# Pivot

In [25]:
# Set 'Year' as the index
result_df.set_index('Year', inplace=True)

# Pivot the DataFrame
pivot = result_df.transpose()

In [26]:
pivot

Year,2018,2019,2021,2022
total_workers,4615506,4662155,4526178,4706068
private_car,90.9,90.7,79.7,81.9
public_transit,1.4,1.4,0.8,0.8
walk,2.3,2.2,1.8,2.0
bike,0.4,0.3,0.3,0.4
taxi_motorcycle_other,0.9,0.9,1.0,1.2
work_from_home,4.1,4.5,16.4,13.7
state,26,26,26,26


# Cleanup

In [27]:
pivot.rename_axis('category', inplace=True)
pivot = pivot.drop(index='state')
pivot = pivot.astype(float)

In [28]:
# Add a new row 'sum' that sums all values within its own column, except for 'total_workers'
pivot.loc['sum_percent'] = pivot[pivot.index != 'total_workers'].sum()

In [29]:
pivot

Year,2018,2019,2021,2022
category,,,,
total_workers,4615506.0,4662155.0,4526178.0,4706068.0
private_car,90.9,90.7,79.7,81.9
public_transit,1.4,1.4,0.8,0.8
walk,2.3,2.2,1.8,2.0
bike,0.4,0.3,0.3,0.4
taxi_motorcycle_other,0.9,0.9,1.0,1.2
work_from_home,4.1,4.5,16.4,13.7
sum_percent,100.0,100.0,100.0,100.0


# Export

In [30]:
pivot.to_csv('./datasets/mi_commuter_methods.csv')